<a href="https://colab.research.google.com/github/nitinsharma006/data_science/blob/main/Neural%20Networks/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning 

<tr>
    <td><img src="images/imagenet_challenge.png" alt="drawing" style="width:500px;" align="left"/></td>
    <td><img src="images/screenshot.png" alt="drawing" style="width:500px;" align="left"/></td>
</tr>

## Classification Problem - Emergency vs Non-emergency Vehicle Classification

<img src='images/emergency_vs_non.png'>


[Source](https://www.slideshare.net/xavigiro/image-classification-on-imagenet-d1l4-2017-upc-deep-learning-for-computer-vision/)

Table of Contents
1. <font color='blue'>Import neccessary libraries
2. Load the data</font>
3. <font color='#999900'>Preprocess the data</font>
4. <font color='red'>Load weights of pre-trained model
5. Fine tune the model for the current problem</font>
6. <font color='blue'>Get predictions</font>

## 1. Import neccessary libraries

Here we will import the required libraries and functions.

In [1]:
# import necessary libraries for plotting images
import numpy as np
import matplotlib.pyplot as plt
# plot in jupyter notebook itself
%matplotlib inline
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from skimage.transform import resize

from tensorflow.keras.utils import to_categorical
#used to preprocess data according to VGG16
from keras.applications.vgg16 import preprocess_input
#for instantiating the model and loading the weights and biases
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, InputLayer

## 2. Load the data

In [3]:
!unzip train_SOaYf6m.zip -d train_SOaYf6m

Archive:  train_SOaYf6m.zip
   creating: train_SOaYf6m/train_SOaYf6m/images/
  inflating: train_SOaYf6m/train_SOaYf6m/images/0.jpg  
  inflating: train_SOaYf6m/train_SOaYf6m/images/1.jpg  
  inflating: train_SOaYf6m/train_SOaYf6m/images/10.jpg  
  inflating: train_SOaYf6m/train_SOaYf6m/images/100.jpg  
  inflating: train_SOaYf6m/train_SOaYf6m/images/1000.jpg  
  inflating: train_SOaYf6m/train_SOaYf6m/images/1001.jpg  
  inflating: train_SOaYf6m/train_SOaYf6m/images/1002.jpg  
  inflating: train_SOaYf6m/train_SOaYf6m/images/1003.jpg  
  inflating: train_SOaYf6m/train_SOaYf6m/images/1004.jpg  
  inflating: train_SOaYf6m/train_SOaYf6m/images/1005.jpg  
  inflating: train_SOaYf6m/train_SOaYf6m/images/1006.jpg  
  inflating: train_SOaYf6m/train_SOaYf6m/images/1007.jpg  
  inflating: train_SOaYf6m/train_SOaYf6m/images/1008.jpg  
  inflating: train_SOaYf6m/train_SOaYf6m/images/1009.jpg  
  inflating: train_SOaYf6m/train_SOaYf6m/images/101.jpg  
  inflating: train_SOaYf6m/train_SOaYf6m/images/

In [4]:
#reading the csv file containing data labels
data = pd.read_csv('train_SOaYf6m/train_SOaYf6m/train.csv')

In [5]:
data.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [6]:
# create an empty python list
X = []

# go through all the image locations one by one
for img_name in data.image_names:
    # read the image from location
    img = plt.imread('train_SOaYf6m/train_SOaYf6m/images/' + img_name)
    # pile it one over the other
    X.append(img)
    
# convert this python list to a single numpy array
X = np.array(X)

In [7]:
#getting the labels for images
y = data.emergency_or_not.values

In [8]:
#converting label to categorical i.e instead of 0/1 labels we have 2 columns emergency and non-emergency ,
#with only one of them is true for every image
y = to_categorical(y)

## 3. Pre-Process Data

Steps : 
1. Pre-process the data as per model's requirement
2. Prepare training and validation set

In [9]:
#show maximum and minimum values for the image array
X.min(), X.max()

(0, 255)

In [11]:
#preprocess input images accordiing to requirements of VGG16 model
X = preprocess_input(X, data_format=None) 

In [12]:
#print minimum and maximum values present in the array
X.min(), X.max()

(-123.68, 151.061)

In [13]:
# splitting the dataset into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

## 4. Load weights of pretrained model

In [14]:
# creating model with pre trained imagenet weights
base_model = VGG16(weights='imagenet')

553476096/553467096 [==============================] - 3s 0us/step


In [15]:
#shows model summary
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

<img src="images/original_vgg_model_input.png" alt="drawing" style="width:500px;"/>

<img src="images/original_vgg_model_knowledge_layer.png" alt="drawing" style="width:500px;"/>

<img src="images/original_vgg_model_top_layer.png" alt="drawing" style="width:500px;"/>

In [16]:
# creating a VGG16 model with imagenet pretrained weights , accepting input of shape (224,224,3)
# also remove the final layers from model(include_top= False)
base_model = VGG16(weights='imagenet', input_shape=(224, 224, 3), include_top=False)

58900480/58889256 [==============================] - 1s 0us/step


In [17]:
# show model summary
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## 5. Fine tune the model for the current problem
Steps:-
1. Extract features
2. Flatten the data
3. Rescale features
4. Create a Neural Network Model
5. Compile the model
6. Train and Validate the model

In [18]:
# extract features using the pretrained VGG16 model
# for training set
base_model_pred = base_model.predict(X_train)
#for validation set
base_model_pred_valid = base_model.predict(X_valid)

In [19]:
#show shape of predictions
base_model_pred.shape

(1152, 7, 7, 512)

In [20]:
# flattening the model output to one dimension for every sample of training set
base_model_pred = base_model_pred.reshape(1152, 7*7*512)

In [21]:
base_model_pred.shape

(1152, 25088)

In [22]:
base_model_pred_valid.shape

(494, 7, 7, 512)

In [23]:
# flattening the model output to one dimension for every sample of validation set
base_model_pred_valid = base_model_pred_valid.reshape(494, 7*7*512)

In [24]:
base_model_pred_valid.shape

(494, 25088)

In [25]:
# checking the min and max of the extracted features
base_model_pred.min(), base_model_pred.max()

(0.0, 499.79898)

In [26]:
#get maximum value from generated features
max_val = base_model_pred.max()

In [27]:
#normalizing features generated from the VGG16 model to [0,1]
base_model_pred = base_model_pred / max_val
base_model_pred_valid = base_model_pred_valid / max_val
base_model_pred.min(), base_model_pred.max()

(0.0, 1.0)

In [28]:
#create a sequential model 
model = Sequential()
# add input layer to the model that accepts input of shape 7*7*512
model.add(InputLayer((7*7*512, )))
# add fully connected layer with 1024 neurons and relu activation
model.add(Dense(units=1024, activation='relu'))
# add fully connected layer with 2 neurons and relu activation
model.add(Dense(units=2, activation='softmax'))

In [29]:
# compile the model
model.compile(optimizer='sgd', metrics=['accuracy'], loss='categorical_crossentropy')

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              25691136  
                                                                 
 dense_1 (Dense)             (None, 2)                 2050      
                                                                 
Total params: 25,693,186
Trainable params: 25,693,186
Non-trainable params: 0
_________________________________________________________________


In [31]:
# train model using features generated from VGG16 model
model.fit(base_model_pred, y_train, epochs=100, validation_data=(base_model_pred_valid, y_valid))

Epoch 1/100
36/36 [==============================] - 5s 126ms/step - loss: 0.6686 - accuracy: 0.5929 - val_loss: 0.6436 - val_accuracy: 0.6599
Epoch 2/100
36/36 [==============================] - 4s 120ms/step - loss: 0.6144 - accuracy: 0.6970 - val_loss: 0.5971 - val_accuracy: 0.7308
Epoch 3/100
36/36 [==============================] - 4s 119ms/step - loss: 0.5665 - accuracy: 0.7830 - val_loss: 0.5520 - val_accuracy: 0.7834
Epoch 4/100
36/36 [==============================] - 4s 122ms/step - loss: 0.5208 - accuracy: 0.8290 - val_loss: 0.5089 - val_accuracy: 0.8097
Epoch 5/100
36/36 [==============================] - 4s 120ms/step - loss: 0.4784 - accuracy: 0.8542 - val_loss: 0.4684 - val_accuracy: 0.8381
Epoch 6/100
36/36 [==============================] - 4s 120ms/step - loss: 0.4401 - accuracy: 0.8776 - val_loss: 0.4329 - val_accuracy: 0.8623
Epoch 7/100
36/36 [==============================] - 4s 122ms/step - loss: 0.4062 - accuracy: 0.8898 - val_loss: 0.4020 - val_accuracy: 0.8664

## 6. Get Predictions

In [33]:
# get predictions
predictions = model.predict(base_model_pred_valid)
#show predictions
predictions

array([[1.13614358e-01, 8.86385620e-01],
       [9.89097238e-01, 1.09026916e-02],
       [9.78099883e-01, 2.19001248e-02],
       [9.95016634e-01, 4.98336181e-03],
       [9.97814894e-01, 2.18509813e-03],
       [9.24264431e-01, 7.57355914e-02],
       [9.92321432e-01, 7.67852226e-03],
       [5.97615028e-04, 9.99402404e-01],
       [7.37034832e-04, 9.99262989e-01],
       [8.00116420e-01, 1.99883595e-01],
       [9.78424549e-01, 2.15754751e-02],
       [9.70308483e-01, 2.96915360e-02],
       [5.63200389e-04, 9.99436796e-01],
       [9.92677927e-01, 7.32210651e-03],
       [4.23617545e-04, 9.99576390e-01],
       [9.69027698e-01, 3.09723299e-02],
       [9.94535446e-01, 5.46453521e-03],
       [1.46247097e-04, 9.99853730e-01],
       [6.88505098e-02, 9.31149423e-01],
       [1.28106266e-01, 8.71893704e-01],
       [9.70201552e-01, 2.97984630e-02],
       [9.94884312e-01, 5.11566550e-03],
       [9.73661840e-01, 2.63380986e-02],
       [9.98200178e-01, 1.79980509e-03],
       [9.969616

## Summary of the video

1. <font color='blue'>Import neccessary libraries
2. Load the data</font>
3. <font color='#999900'>Preprocess the data</font>
4. <font color='red'>Load weights of pre-trained model
5. Fine tune the model for the current problem</font>
6. <font color='blue'>Get predictions</font>